In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

from calitp_data_analysis import get_fs
from calitp_data_analysis.tables import tbls
import calitp_data_analysis.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# Updating procedure for updating Conveyal network bundles

* much of this could be in DBT?

In [47]:
target_date = dt.date(2023, 8, 30)

In [48]:
feeds_on_target = shared_utils.gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(selected_date=target_date)
# default will use mtc subfeeds (prev Conveyal behavior), can spec customer facing if we wanna switch

In [49]:
feeds_on_target >> head(3)

,key,date,feed_key,feed_timezone,base64_url,gtfs_dataset_key,gtfs_dataset_name,name,regional_feed_type,type
0,976d47278b5ef81c33c7e7564e05c325,2023-08-30,e988dc0748d7a4533481e55ad98c9cf2,America/Los_Angeles,aHR0cHM6Ly90Y3J0YS50cmlwc2hvdC5jb20vdjEvZ3Rmcy...,0139b1253130b33adcd4b3a4490530d2,TCRTA TripShot Schedule,TCRTA TripShot Schedule,None,schedule
1,4a3f91a05acc9f21377ee732a62d97f3,2023-08-30,4ca29da6f3017b5f6cacf687e0572543,America/Los_Angeles,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,015d67d5b75b5cf2b710bbadadfb75f5,Bay Area 511 Marin Schedule,Bay Area 511 Marin Schedule,Regional Subfeed,schedule
2,2e19a204d9c5d723bef5d31979c6a535,2023-08-30,5e1be3854cae4470eccf4a1323526e3a,America/Los_Angeles,aHR0cHM6Ly9naXRodWIuY29tL0xBQ01UQS9sb3MtYW5nZW...,04540587c205fb3b1a86b76badacb919,Playa Vista Schedule,Playa Vista Schedule,None,schedule


In [56]:
feeds_on_target.shape

(204, 10)

In [51]:
metro_test = (feeds_on_target >> filter(_.gtfs_dataset_name.str.contains('LA Metro'))).base64_url.iloc[0]

## Stops --> Region matching

In [7]:
regions = {}
# Set bounds for northern california
regions['norcal'] = {'north': 42.03909, 'south': 39.07038, 'east': -119.60541, 'west': -124.49158}
regions['central'] = {'north': 39.64165, 'south': 35.87347, 'east': -117.53174, 'west': -123.83789}
regions['socal'] = {'north': 35.8935, 'south': 32.5005, 'east': -114.13121, 'west': -121.46759}
regions['mojave'] = {'north': 37.81629, 'south': 34.89945, 'east': -114.59015, 'west': -118.38043}

In [8]:
import shapely

In [45]:
# https://shapely.readthedocs.io/en/stable/reference/shapely.box.html#shapely.box
# xmin, ymin, xmax, ymax
to_bbox = lambda x: [x['west'], x['south'], x['east'], x['north']]

In [38]:
df = pd.DataFrame(regions).transpose().reset_index().rename(columns={'index':'region'})

In [39]:
df['bbox'] = df.apply(to_bbox, axis=1)

In [40]:
df

,region,north,south,east,west,bbox
0,norcal,42.03909,39.07038,-119.60541,-124.49158,"[-124.49158, 39.07038, -119.60541, 42.03909]"
1,central,39.64165,35.87347,-117.53174,-123.83789,"[-123.83789, 35.87347, -117.53174, 39.64165]"
2,socal,35.89350,32.50050,-114.13121,-121.46759,"[-121.46759, 32.5005, -114.13121, 35.8935]"
3,mojave,37.81629,34.89945,-114.59015,-118.38043,"[-118.38043, 34.89945, -114.59015, 37.81629]"


In [42]:
df['geometry'] = df.apply(lambda x: shapely.geometry.box(*x.bbox), axis = 1)

In [43]:
gdf = gpd.GeoDataFrame(df, crs=shared_utils.geography_utils.WGS84)

In [46]:
# gdf.explore()

## Validation steps here!

In [52]:
validation_tbl = tbls.mart_gtfs_quality.fct_daily_schedule_feed_validation_notices()

In [74]:
# confirm no errors in source table -- check reports?

# smart = (feeds_on_target >> filter(_.gtfs_dataset_name.str.contains('Sono'))).base64_url.iloc[0]

# ## apparently busted since moving to v4.1 on Sep 1??
# validation_df = (validation_tbl >> filter(_.date == '2023-07-18')
#                                 >> filter(_.total_notices > 0)
#                                 >> filter(_.severity == 'ERROR')
#                                 >> filter(_.base64_url == smart)
#                                 # >> distinct(_.date, _.base64_url, _.severity)
#                             ) >> collect()

# validation_df

In [82]:
## apparently busted since moving to v4.1 on Sep 1??
validation_df = (validation_tbl >> filter(_.date == target_date)
                                >> filter(_.severity == 'ERROR',
                                         _.total_notices > 0)
                                >> distinct(_.date, _.base64_url, _.severity,
                                            _.total_notices)
                            ) >> collect()

In [83]:
validation_df

,date,base64_url,severity,total_notices
0,2023-08-30,aHR0cHM6Ly9ndGZzLmNhbGl0cC5vcmcvcHJvZHVjdGlvbi...,ERROR,20
1,2023-08-30,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,ERROR,3
2,2023-08-30,aHR0cHM6Ly9ndGZzLmNhbGl0cC5vcmcvcHJvZHVjdGlvbi...,ERROR,12
3,2023-08-30,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,ERROR,612
4,2023-08-30,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,ERROR,8
...,...,...,...,...
109,2023-08-30,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,ERROR,46
110,2023-08-30,aHR0cHM6Ly9ndGZzLmNhbGl0cC5vcmcvcHJvZHVjdGlvbi...,ERROR,24
111,2023-08-30,aHR0cHM6Ly9naXRodWIuY29tL0xBQ01UQS9sb3MtYW5nZW...,ERROR,6
112,2023-08-30,aHR0cHM6Ly9ndGZzLmNhbGl0cC5vcmcvcHJvZHVjdGlvbi...,ERROR,28


In [84]:
with_notices = feeds_on_target >> left_join(_, validation_df, on =['base64_url', 'date'])

In [86]:
with_notices >> arrange(-_.total_notices) >> head(5)

,key,date,feed_key,feed_timezone,base64_url,gtfs_dataset_key,gtfs_dataset_name,name,regional_feed_type,type,severity,total_notices
222,5a944928eeda308d3b7df42e5ef5ae99,2023-08-30,55407fafa4579761cd7481c32f0d73df,America/Los_Angeles,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,e524db270831632bdcf71df1d7e74d25,Lassen Schedule,Lassen Schedule,None,schedule,ERROR,3284.0
205,9f9cd105ae8cdc6467c7eacbe3a0ae2e,2023-08-30,fe662c95bddfb6e5fd75cb0afbb85cd8,America/Los_Angeles,aHR0cHM6Ly9ndGZzLmNhbGl0cC5vcmcvcHJvZHVjdGlvbi...,cde2b7a63ab7bb33141c8b02b001ea0f,TCAT Flex,TCAT Flex,None,schedule,ERROR,3105.0
206,9f9cd105ae8cdc6467c7eacbe3a0ae2e,2023-08-30,fe662c95bddfb6e5fd75cb0afbb85cd8,America/Los_Angeles,aHR0cHM6Ly9ndGZzLmNhbGl0cC5vcmcvcHJvZHVjdGlvbi...,cde2b7a63ab7bb33141c8b02b001ea0f,TCAT Flex,TCAT Flex,None,schedule,ERROR,3099.0
104,2aba7b8a16754bcf57ec6d58ed907c13,2023-08-30,c14e920eaf0dba1815ec801c4ac51775,America/Los_Angeles,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,6fa88692b54f9ce98d2b95ae6804fa43,Merced Schedule,Merced Schedule,None,schedule,ERROR,2292.0
95,94a09768998b6c79c22abaebb65edfde,2023-08-30,598ff5ab409c5158804e68abfc5698fa,America/Los_Angeles,aHR0cHM6Ly9kYXRhLnRyaWxsaXVtdHJhbnNpdC5jb20vZ3...,6a74a120fe2da801e778ac523b354e2b,Trinity Schedule,Trinity Schedule,None,schedule,ERROR,2069.0


## Download raw from GCS (for Conveyal)

In [8]:
fs = get_fs()

In [30]:
base64_url = metro_test

In [31]:
# need wildcard for file too -- not all are gtfs.zip!
uri = f'gs://calitp-gtfs-schedule-raw-v2/schedule/dt={target_date}/*/base64_url={base64_url}/*.zip'

In [32]:
fs.get(uri, './gtfs.zip')

[None]

In [ ]:
def 